### Format and clean the downloaded (through API) DGS interval data

##### For hourly aggregation, we need to right align the 1hr window, and then sum the killo watt hours
```
id	Date	Time	kWh
1	11/7/2016	0:15:00	64.01
1	11/7/2016	0:30:00	63.5
1	11/7/2016	0:45:00	63.34
1	11/7/2016	1:00:00	64.01
---------------------------------
1	11/7/2016	1:15:00	64.21
1	11/7/2016	1:30:00	63.35
1	11/7/2016	1:45:00	63.27
1	11/7/2016	2:00:00	63.56
```
#### Aggregated values are
```
1	11/7/2016	1:00:00	254.86
1	11/7/2016	2:00:00	254.39
```

In [393]:
import os
import json
import requests
import string
from datetime import datetime
import pytz
import pickle
import numpy as np
import pandas as pd

In [394]:
raw_dir        = "../data/raw/"
raw_dir1       = "../data/raw/dgs_15min_api/"
processed_dir  = "../data/raw/dgs_15min_api_1hr_agr_all/"
processed_dir1 = "../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered/"
processed_dir2 = "../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered_with_na/"
pkl_filename   = "../data/raw/dgs_15min_api_all.pkl"

if not os.path.exists(processed_dir):
    os.makedirs(processed_dir)
if not os.path.exists(processed_dir1):
    os.makedirs(processed_dir1)    
if not os.path.exists(processed_dir2):
    os.makedirs(processed_dir1)        

In [ ]:
# def load_all_data(directory):
#     ### Load all data -- this will take a lot of time
#     dfa = {}
#     filenames = os.listdir(raw_dir1)
#     for filename in filenames:    
#         if filename.endswith('.csv'):
#             bid = int(filename.split(".")[0])
#             df = pd.read_csv(os.path.join(raw_dir1, filename), index_col=0, parse_dates=[0])        
#             #df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y-%m-%d %H:%M')        
#             #.tz_localize('UTC').tz_convert('US/Eastern')
#             #df.set_index('timestamp', inplace=True)
#             df.drop(['epoch'], inplace=True, axis=1)        
#             df.index = df.index.tz_localize('UTC').tz_convert('US/Eastern')
#             dfa[bid] = df
#             print '%3d  %6d  %s - %s  %5d' % (bid, len(df), df.index[0].date(), df.index[len(df)-1].date(), df['kWh'].isnull().sum())        
#     return dfa            
        
# with open(pkl_filename, 'wb') as output:
#     pickle.dump(dfa, output, pickle.HIGHEST_PROTOCOL)       
    
# with open(pkl_filename, 'rb') as input:
#     dfall = pickle.load(input)
# #dfall1.keys()        

##### Aggregate (sum) to 1 hour interval readings
Since we have kilo-watt-hour readings at 15-min interval, we have to SUM all 4 values to get kilo-watt-hour for 1 hour. In case, a 1hour time-window has less than 4 valid values, then NA is filled up for the same. 

In [397]:
dfall_1hr = {}
dfm = pd.DataFrame(columns=('start_date', 'end_date', 'total', 'missing', 'valid', 'consecutive_na'))

print 'Started at ', datetime.now()
filenames = os.listdir(raw_dir1)
for filename in filenames:    
    if not filename.endswith('.csv'):
        continue
        
    bid = int(filename.split(".")[0])

    df = pd.read_csv(os.path.join(raw_dir1, filename), index_col=0, parse_dates=[0])
    df.drop(['epoch'], inplace=True, axis=1)
    df.index = df.index.tz_localize('UTC').tz_convert('US/Eastern')

    df = df.resample('1H', closed='right', label='right', convention='end', kind='timestamp').sum(min_count=4)
    df.kWh = df.kWh.round(2)

    filename = os.path.join(processed_dir, str(bid) + '.csv')
    df.to_csv(filename)
    print '%3d  %6d %s' % (bid, len(df), filename)
    dfall_1hr[bid] = df
    
    start_date = df.index[0].date()    
    end_date   = df.index[len(df)-1].date()    
    total      = len(df)
    missing    = pd.isnull(df['kWh']).sum()
    valid      = total - missing
    consecutive_na = sum(df.kWh.isnull().astype(int).groupby(df.kWh.notnull().astype(int).cumsum()).sum())
    dfm.loc[bid] = [start_date, end_date, total, missing, valid, consecutive_na]
    
print 'Ended at ', datetime.now()    

dfm.index.name = 'id'    
filename = os.path.join(raw_dir, 'dgs_15min_api_1hr_agr_missing_values.csv')
dfm.to_csv(filename)
print dfm.shape
dfm.head()    

  1   47640 ../data/raw/dgs_15min_api_1hr_agr_all/1.csv
 10   24331 ../data/raw/dgs_15min_api_1hr_agr_all/10.csv
100   47640 ../data/raw/dgs_15min_api_1hr_agr_all/100.csv
101   47640 ../data/raw/dgs_15min_api_1hr_agr_all/101.csv
102   47640 ../data/raw/dgs_15min_api_1hr_agr_all/102.csv
103   47640 ../data/raw/dgs_15min_api_1hr_agr_all/103.csv
104   47640 ../data/raw/dgs_15min_api_1hr_agr_all/104.csv
105   47640 ../data/raw/dgs_15min_api_1hr_agr_all/105.csv
106   47640 ../data/raw/dgs_15min_api_1hr_agr_all/106.csv
107   47640 ../data/raw/dgs_15min_api_1hr_agr_all/107.csv
108   47640 ../data/raw/dgs_15min_api_1hr_agr_all/108.csv
109   47640 ../data/raw/dgs_15min_api_1hr_agr_all/109.csv
 11   47640 ../data/raw/dgs_15min_api_1hr_agr_all/11.csv
110   47640 ../data/raw/dgs_15min_api_1hr_agr_all/110.csv
111   56952 ../data/raw/dgs_15min_api_1hr_agr_all/111.csv
112   47640 ../data/raw/dgs_15min_api_1hr_agr_all/112.csv
113   47640 ../data/raw/dgs_15min_api_1hr_agr_all/113.csv
114   47640 ../dat

229   47640 ../data/raw/dgs_15min_api_1hr_agr_all/229.csv
 23   47640 ../data/raw/dgs_15min_api_1hr_agr_all/23.csv
230   47640 ../data/raw/dgs_15min_api_1hr_agr_all/230.csv
231   47640 ../data/raw/dgs_15min_api_1hr_agr_all/231.csv
232   47640 ../data/raw/dgs_15min_api_1hr_agr_all/232.csv
233   15601 ../data/raw/dgs_15min_api_1hr_agr_all/233.csv
234   27696 ../data/raw/dgs_15min_api_1hr_agr_all/234.csv
235   47640 ../data/raw/dgs_15min_api_1hr_agr_all/235.csv
236   47640 ../data/raw/dgs_15min_api_1hr_agr_all/236.csv
237   27049 ../data/raw/dgs_15min_api_1hr_agr_all/237.csv
238   47640 ../data/raw/dgs_15min_api_1hr_agr_all/238.csv
239   47640 ../data/raw/dgs_15min_api_1hr_agr_all/239.csv
 24   47640 ../data/raw/dgs_15min_api_1hr_agr_all/24.csv
240   24337 ../data/raw/dgs_15min_api_1hr_agr_all/240.csv
241   47640 ../data/raw/dgs_15min_api_1hr_agr_all/241.csv
242   32832 ../data/raw/dgs_15min_api_1hr_agr_all/242.csv
243   31320 ../data/raw/dgs_15min_api_1hr_agr_all/243.csv
244   42673 ../d

391   45985 ../data/raw/dgs_15min_api_1hr_agr_all/391.csv
395    9865 ../data/raw/dgs_15min_api_1hr_agr_all/395.csv
397   47640 ../data/raw/dgs_15min_api_1hr_agr_all/397.csv
398   47640 ../data/raw/dgs_15min_api_1hr_agr_all/398.csv
399   47640 ../data/raw/dgs_15min_api_1hr_agr_all/399.csv
  4   46680 ../data/raw/dgs_15min_api_1hr_agr_all/4.csv
 40   47640 ../data/raw/dgs_15min_api_1hr_agr_all/40.csv
400   47256 ../data/raw/dgs_15min_api_1hr_agr_all/400.csv
401   41713 ../data/raw/dgs_15min_api_1hr_agr_all/401.csv
403   14138 ../data/raw/dgs_15min_api_1hr_agr_all/403.csv
404   47640 ../data/raw/dgs_15min_api_1hr_agr_all/404.csv
405   47640 ../data/raw/dgs_15min_api_1hr_agr_all/405.csv
407   47640 ../data/raw/dgs_15min_api_1hr_agr_all/407.csv
408   47640 ../data/raw/dgs_15min_api_1hr_agr_all/408.csv
 41   47640 ../data/raw/dgs_15min_api_1hr_agr_all/41.csv
411   47640 ../data/raw/dgs_15min_api_1hr_agr_all/411.csv
413   34753 ../data/raw/dgs_15min_api_1hr_agr_all/413.csv
414   27097 ../dat

596   45745 ../data/raw/dgs_15min_api_1hr_agr_all/596.csv
597   47640 ../data/raw/dgs_15min_api_1hr_agr_all/597.csv
599   43369 ../data/raw/dgs_15min_api_1hr_agr_all/599.csv
  6   47544 ../data/raw/dgs_15min_api_1hr_agr_all/6.csv
 60   45817 ../data/raw/dgs_15min_api_1hr_agr_all/60.csv
600   29248 ../data/raw/dgs_15min_api_1hr_agr_all/600.csv
601   47640 ../data/raw/dgs_15min_api_1hr_agr_all/601.csv
602    9433 ../data/raw/dgs_15min_api_1hr_agr_all/602.csv
603   10081 ../data/raw/dgs_15min_api_1hr_agr_all/603.csv
604   39035 ../data/raw/dgs_15min_api_1hr_agr_all/604.csv
605   12846 ../data/raw/dgs_15min_api_1hr_agr_all/605.csv
606   35953 ../data/raw/dgs_15min_api_1hr_agr_all/606.csv
608   47640 ../data/raw/dgs_15min_api_1hr_agr_all/608.csv
 61   47640 ../data/raw/dgs_15min_api_1hr_agr_all/61.csv
610   24961 ../data/raw/dgs_15min_api_1hr_agr_all/610.csv
612   47640 ../data/raw/dgs_15min_api_1hr_agr_all/612.csv
613   47640 ../data/raw/dgs_15min_api_1hr_agr_all/613.csv
614   46920 ../dat

,start_date,end_date,total,missing,valid,consecutive_na
id,,,,,,
1,2013-01-23,2018-07-01,47640,315,47325,315
10,2013-01-23,2015-11-02,24331,263,24068,263
100,2013-01-23,2018-07-01,47640,508,47132,508
101,2013-01-23,2018-07-01,47640,267,47373,267
102,2013-01-23,2018-07-01,47640,355,47285,355


In [ ]:
# bid = '1'
# filename = bid + '.csv'
# print raw_dir1, bid
# df = pd.read_csv(os.path.join(raw_dir1, filename), index_col=0, parse_dates=[0])        
# df.drop(['epoch'], inplace=True, axis=1)        
# df.index = df.index.tz_localize('UTC').tz_convert('US/Eastern')

# df = df.resample('1H', closed='right', label='right', convention='end', kind='timestamp').sum(min_count=4)
# df.kWh = df.kWh.round(2)

# df1 = df.copy()
# df1.index = df1.index.tz_localize(None)
# df1.index.freq = None
# df1.head()

##### Filling missing values
Following rules are applied sequentially until filling the missing value

1. Use the same-hour previous day value if that value is a original reading (not a filled up value)
2. Use the same-hour next     day value if that value is a original reading (not a filled up value)
3. Use the previous-hour value if that value is a original reading (not a filled up value)
4. Use the next-hour     value if that value is a original reading (not a filled up value)

If none of the above rules are satisfied, then we don't fill the value.

Above rules are derived after exploring the missing value patterns in each building. Most common missing values are 
1. Missing value for 1 or few hours - Fill it up using previous/next day same-hour value or prev/next hour value
2. Missing values for 1-2 full days - Fill it up using previous/next day values. Maximum of TWO duplicate days
3. Missing values for 3 days or more - Fill it up using previous/next day values. Maximum of TWO duplicate days, one day after the last valid values and one day before the valid values day

Please note that, we don't fill up the missing datetime intervals, including day-light saving hours. 

In [396]:
# filling missing values
def fill_na_prev_day_new(df):
    #df = df.drop_duplicates()
    #df = df.drop_duplicates(subset='rownum', keep='last')
    #remove duplicate index
    #df = df[~df.index.duplicated()]
    telda_1day = pd.Timedelta(days=1)
    telda_1hr  = pd.Timedelta(hours=1)
    df["filled"] = np.nan
    ix = 0
    
    df1 = df.loc[pd.isna(df['kWh'])]
    print '\tno. missing values ', df1.shape
    
    for index, row in df1.iterrows():        
        #print 'checking ', index, type(index)
        #print 'value ', df.loc[index, 'kWh']
        
        prev_day = index - telda_1day
        next_day = index + telda_1day

        prev_hr = index - telda_1hr
        next_hr = index + telda_1hr
        #print 'filling ', index, df.loc[index, 'kWh'], df.loc[index, 'filled'], pd.isna(df.loc[index, 'filled'])
        #print '\tprev_day in df.index ', prev_day, prev_day in df.index

        if prev_day in df.index:
            kwh = df.loc[prev_day, 'kWh']
            fil = df.loc[prev_day, 'filled']                
            if not pd.isna(kwh) and pd.isna(fil):
                df.loc[index, 'kWh']    = df.loc[prev_day, 'kWh']
                df.loc[index, 'filled'] = True
                continue


        if next_day in df.index:
            kwh = df.loc[next_day, 'kWh']
            fil = df.loc[next_day, 'filled']                
            if not pd.isna(kwh) and pd.isna(fil):
                df.loc[index, 'kWh']    = df.loc[next_day, 'kWh']
                df.loc[index, 'filled'] = True
                continue

        if prev_hr in df.index:
            kwh = df.loc[prev_hr, 'kWh']
            fil = df.loc[prev_hr, 'filled']                
            if not pd.isna(kwh) and pd.isna(fil):
                df.loc[index, 'kWh']    = df.loc[prev_hr, 'kWh']
                df.loc[index, 'filled'] = True
                continue

        if next_hr in df.index:
            kwh = df.loc[next_hr, 'kWh']
            fil = df.loc[next_hr, 'filled']                
            if not pd.isna(kwh) and pd.isna(fil):
                df.loc[index, 'kWh']    = df.loc[next_hr, 'kWh']
                df.loc[index, 'filled'] = True
                continue                    
            #else: 
            #    #print 'filling ', index, ' using -- None'
                None
        #drop all NAs in the begining and end
        df = df.loc[df.index >= df.first_valid_index()]
        df = df.loc[df.index <= df.last_valid_index() ]
    return df

In [ ]:
# print datetime.now()
# df1 = df1[~df1.index.duplicated()]

# df1_f = fill_na_prev_day_new(df1)

# print df1_f.shape
# df1_f = df1_f[np.isfinite(df1_f['kWh'])]
# print df1_f.shape
# df1_f.to_csv(os.path.join(raw_dir, 'group1.csv'))

# print datetime.now()

#### Aggregate and clean the data

In [279]:
dfall_1hr_cleaned = {}
dfall_1hr_cleaned_all = {}

filenames = os.listdir(raw_dir1)
for filename in filenames:    
    if not filename.endswith('.csv'):
        continue
        
    try:                
        print datetime.now(), filename
        bid = int(filename.split(".")[0])

        df = pd.read_csv(os.path.join(raw_dir1, filename), index_col=0, parse_dates=[0])
        df.drop(['epoch'], inplace=True, axis=1)
        df.index = df.index.tz_localize('UTC').tz_convert('US/Eastern')

        df = df.resample('1H', closed='right', label='right', convention='end', kind='timestamp').sum(min_count=4)
        df.kWh = df.kWh.round(2)

        df1 = df.copy()
        df1.index = df1.index.tz_localize(None)
        df1.index.freq = None

        #remove duplicates
        df1 = df1[~df1.index.duplicated()]

        df2 = fill_na_prev_day_new(df1)
        df3 = df2 # fill_na_prev_day_new(df2)

        dfall_1hr_cleaned_all[bid] = df3
        filename = os.path.join(processed_dir2, str(bid) + '.csv')
        df3.to_csv(filename)
        print '\t', filename, df3.shape

        df4 = df3.copy()
        # drop the remaining missing na rows
        df4 = df4[np.isfinite(df4['kWh'])]
        filename = os.path.join(processed_dir1, str(bid) + '.csv')
        df4.to_csv(filename)
        dfall_1hr_cleaned[bid] = df4

        print '\t', filename, df4.shape
    except Exception as e:
        print 'Exception ' + str(e)

2018-07-16 23:03:36.345000 1.csv
	no. missing values  (315, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered_with_na/1.csv (47635, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered/1.csv (47610, 2)
2018-07-16 23:03:42.614000 10.csv
	no. missing values  (263, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered_with_na/10.csv (24328, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered/10.csv (24303, 2)
2018-07-16 23:03:46.646000 100.csv
	no. missing values  (508, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered_with_na/100.csv (47635, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered/100.csv (47561, 2)
2018-07-16 23:03:54.587000 101.csv
	no. missing values  (267, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered_with_na/101.csv (47635, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered/101.csv (47610, 2)
2018-07-16 23:04:00.924000 102.csv
	no. missing values  (355, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered_with_

	no. missing values  (246, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered_with_na/132.csv (19222, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered/132.csv (19174, 2)
2018-07-16 23:19:14.267000 133.csv
	no. missing values  (290, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered_with_na/133.csv (47635, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered/133.csv (47610, 2)
2018-07-16 23:19:20.556000 134.csv
	no. missing values  (340, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered_with_na/134.csv (47635, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered/134.csv (47610, 2)
2018-07-16 23:19:27.676000 135.csv
	no. missing values  (463, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered_with_na/135.csv (47635, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered/135.csv (47610, 2)
2018-07-16 23:19:35.806000 136.csv
	no. missing values  (362, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered_with_na/136.csv (44852, 2)
	../

	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered_with_na/167.csv (47635, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered/167.csv (47610, 2)
2018-07-16 23:30:16.747000 168.csv
	no. missing values  (314, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered_with_na/168.csv (47611, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered/168.csv (47586, 2)
2018-07-16 23:30:22.964000 169.csv
	no. missing values  (195, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered_with_na/169.csv (47635, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered/169.csv (47610, 2)
2018-07-16 23:30:28.603000 17.csv
	no. missing values  (254, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered_with_na/17.csv (23182, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered/17.csv (23157, 2)
2018-07-16 23:30:32.065000 170.csv
	no. missing values  (1088, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered_with_na/170.csv (22798, 2)
	../data/raw/dgs_15min_api_1hr_agr_f

	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered/20.csv (47610, 2)
2018-07-16 23:37:35.240000 200.csv
	no. missing values  (240, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered_with_na/200.csv (47635, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered/200.csv (47610, 2)
2018-07-16 23:37:40.497000 201.csv
	no. missing values  (241, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered_with_na/201.csv (47635, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered/201.csv (47610, 2)
2018-07-16 23:37:45.600000 202.csv
	no. missing values  (315, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered_with_na/202.csv (47635, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered/202.csv (47610, 2)
2018-07-16 23:37:51.382000 203.csv
	no. missing values  (266, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered_with_na/203.csv (47635, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered/203.csv (47610, 2)
2018-07-16 23:37:57.446000 204.csv
	no. mis

	no. missing values  (362, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered_with_na/235.csv (47635, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered/235.csv (47610, 2)
2018-07-17 00:06:38.937000 236.csv
	no. missing values  (649, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered_with_na/236.csv (47635, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered/236.csv (47249, 2)
2018-07-17 00:06:51.076000 237.csv
	no. missing values  (337, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered_with_na/237.csv (27046, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered/237.csv (27021, 2)
2018-07-17 00:06:54.900000 238.csv
	no. missing values  (364, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered_with_na/238.csv (47635, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered/238.csv (47610, 2)
2018-07-17 00:07:00.363000 239.csv
	no. missing values  (363, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered_with_na/239.csv (47635, 2)
	../

...

	no. missing values  (23502, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered_with_na/467.csv (47635, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered/467.csv (24332, 2)
2018-07-17 01:11:27.416000 468.csv
	no. missing values  (130, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered_with_na/468.csv (34749, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered/468.csv (34749, 2)
2018-07-17 01:11:31.470000 469.csv
	no. missing values  (313, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered_with_na/469.csv (45620, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered/469.csv (45620, 2)
2018-07-17 01:11:36.691000 47.csv
	no. missing values  (338, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered_with_na/47.csv (47635, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered/47.csv (47610, 2)
2018-07-17 01:11:43.307000 470.csv
	no. missing values  (218, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered_with_na/470.csv (47443, 2)
	../d

	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered_with_na/517.csv (47635, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered/517.csv (47610, 2)
2018-07-17 01:21:25.932000 518.csv
	no. missing values  (5823, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered_with_na/518.csv (47635, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered/518.csv (42260, 2)
2018-07-17 01:23:31.850000 52.csv
	no. missing values  (289, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered_with_na/52.csv (47635, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered/52.csv (47610, 2)
2018-07-17 01:23:38.312000 520.csv
	no. missing values  (282, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered_with_na/520.csv (46100, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered/520.csv (46100, 2)
2018-07-17 01:23:43.629000 522.csv
	no. missing values  (12328, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered_with_na/522.csv (56850, 2)
	../data/raw/dgs_15min_api_1hr_agr

	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered/574.csv (47610, 2)
2018-07-17 01:33:53.616000 575.csv
	no. missing values  (26281, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered_with_na/575.csv (13678, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered/575.csv (13558, 2)
2018-07-17 01:37:03.057000 576.csv
	no. missing values  (2022, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered_with_na/576.csv (42140, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered/576.csv (40975, 2)
2018-07-17 01:37:35.115000 578.csv
	no. missing values  (378, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered_with_na/578.csv (47635, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered/578.csv (47610, 2)
2018-07-17 01:37:41.966000 579.csv
	no. missing values  (355, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered_with_na/579.csv (47635, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered/579.csv (47610, 2)
2018-07-17 01:37:48.557000 58.csv
	no. 

	no. missing values  (173, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered_with_na/626.csv (47635, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered/626.csv (47610, 2)
2018-07-17 01:42:50.637000 627.csv
	no. missing values  (54, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered_with_na/627.csv (23950, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered/627.csv (23950, 2)
2018-07-17 01:42:53.645000 628.csv
	no. missing values  (79, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered_with_na/628.csv (23950, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered/628.csv (23950, 2)
2018-07-17 01:42:56.398000 629.csv
	no. missing values  (79, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered_with_na/629.csv (23950, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered/629.csv (23950, 2)
2018-07-17 01:42:59.155000 63.csv
	no. missing values  (482, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered_with_na/63.csv (47635, 2)
	../data/

	no. missing values  (214, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered_with_na/9.csv (30789, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered/9.csv (30764, 2)
2018-07-17 01:56:16.883000 90.csv
	no. missing values  (548, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered_with_na/90.csv (12839, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered/90.csv (12502, 2)
2018-07-17 01:56:22.211000 91.csv
	no. missing values  (116, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered_with_na/91.csv (26518, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered/91.csv (26493, 2)
2018-07-17 01:56:25.600000 92.csv
	no. missing values  (288, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered_with_na/92.csv (47635, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered/92.csv (47610, 2)
2018-07-17 01:56:32.101000 93.csv
	no. missing values  (457, 2)
	../data/raw/dgs_15min_api_1hr_agr_filled_and_filtered_with_na/93.csv (47635, 2)
	../data/raw/dgs_15

#### Make dataset summary

In [391]:
dfmeta = pd.DataFrame(columns=('start_date', 'end_date', 'total', 'filled'))
yr_count = []

for bid, df in dfall_1hr_cleaned.iteritems():
    #if bid > 12:
    #    break
    #print bid, df.shape
    start_date = df.index[0].date()    
    end_date   = df.index[len(df)-1].date()    
    total      = len(df)
    filled     = total - pd.isnull(df['filled']).sum()    
    dfmeta.loc[bid] = [start_date, end_date, total, filled]
    print '%3d  %s %s %6d %4d' % (bid, start_date, end_date, len(df), filled)
    
    cf = df.groupby(df.index.year).kWh.count()
    cf.name = str(bid)    
    yr_count.insert(0, cf)

dfmeta.index.name = 'id'    

df_yr_count = pd.concat(yr_count, axis=1).transpose()
df_yr_count.index.name = 'id'    
df_yr_count.index = pd.to_numeric(df_yr_count.index)

dfmeta = dfmeta.join(df_yr_count, how='outer')

filename = os.path.join(raw_dir, 'dgs_15min_api_1hr_agr_filled_and_filtered_summary.csv')
dfmeta.to_csv(filename, float_format = '%.0f')
dfmeta.head()

  1  2013-01-23 2018-07-01  47610  290
  2  2013-04-03 2018-07-01  45932  377
  3  2013-01-23 2018-06-28  47549  474
  4  2013-03-04 2018-07-01  46507  252
  5  2013-01-23 2018-07-01  47230  204
  6  2013-01-23 2018-06-27  47514  129
  7  2013-01-23 2018-06-27  47514  118
  8  2013-01-23 2018-07-01  28739  386
  9  2013-01-23 2016-07-29  30764  189
 10  2013-01-23 2015-11-02  24303  238
 11  2013-01-23 2018-07-01  47610  168
 12  2013-01-23 2016-05-01  28628   92
 13  2013-01-23 2018-06-06  46804  314
 14  2013-01-23 2017-03-31  36643  142
 15  2013-05-23 2018-07-01  44588  294
 16  2013-01-23 2018-07-01  47610  240
 17  2013-01-23 2015-09-16  23157  229
 18  2013-05-01 2016-12-07  31582  215
 19  2013-04-04 2018-07-01  45932  192
 20  2013-01-23 2018-07-01  47610  314
 21  2016-02-04 2018-07-01  21070   53
 22  2013-01-23 2018-07-01  31698  285
 23  2013-01-23 2018-07-01  47610  144
 24  2013-01-23 2018-07-01  47610  193
 25  2013-03-29 2015-11-24  23159  500
 26  2013-01-23 2018-07-0

219  2013-01-23 2018-07-01  47478  479
220  2013-01-23 2018-07-01  47610  312
221  2013-01-23 2018-07-01  47610  325
222  2013-01-23 2018-07-01  47610  218
223  2013-01-23 2018-07-01  47610  337
224  2013-01-23 2018-07-01  47610  284
225  2013-01-23 2018-07-01  47610  240
226  2013-01-23 2018-07-01  47610  338
227  2013-01-23 2018-07-01  47610  239
228  2013-02-15 2018-07-01  47083  211
229  2013-01-23 2018-07-01  47610  145
230  2013-01-23 2018-07-01  47610  290
231  2013-01-23 2018-07-01  47610  362
232  2013-01-23 2018-07-01  47610  217
233  2013-01-23 2014-11-04  15574  307
234  2013-01-23 2016-03-22  27668  140
235  2013-01-23 2018-07-01  47610  337
236  2013-01-23 2018-07-01  47249  263
237  2013-01-23 2016-02-24  27021  312
238  2013-01-23 2018-07-01  47610  339
239  2013-01-23 2018-07-01  47610  338
240  2013-01-23 2015-11-03  24309  139
241  2013-01-23 2018-07-01  47610  266
242  2013-01-23 2016-10-22  32804  165
243  2013-01-23 2016-08-20  31292  166
244  2013-08-18 2018-07-0

530  2013-01-23 2016-10-11  32539  200
531  2014-08-02 2018-07-01  34293  105
533  2013-01-23 2018-07-01  47610  283
535  2013-01-23 2018-07-01  47598  166
539  2012-01-01 2018-07-01  55621  430
540  2015-08-25 2016-12-16  10368  100
541  2013-01-23 2018-07-01  47609  331
545  2013-01-23 2018-07-01  47610  260
547  2013-01-23 2017-06-19  38563  226
550  2013-01-23 2018-07-01  47610  216
551  2013-01-23 2018-07-01  47610  289
552  2013-01-23 2018-07-01  47610  279
555  2013-01-23 2015-09-11  23037  254
556  2013-01-23 2018-07-01  47586  326
557  2013-01-23 2014-05-17  11470  125
558  2013-01-23 2014-05-17  11470  125
559  2013-04-15 2014-05-17   9528   90
561  2013-01-23 2018-07-01  47154  974
567  2015-10-07 2018-07-01  23950  104
568  2013-06-18 2018-07-01  41924  560
570  2013-03-29 2018-07-01  46076  163
571  2013-03-29 2018-07-01  46075  302
572  2013-01-23 2018-07-01  47610  332
574  2013-01-23 2018-07-01  47610  338
575  2016-12-08 2018-07-01  13558  243
576  2013-09-09 2018-07-0

,start_date,end_date,total,filled,2012,2013,2014,2015,2016,2017,2018
id,,,,,,,,,,,
1,2013-01-23,2018-07-01,47610,290,NaN,8206.0,8759.0,8759.0,8783.0,8759.0,4344.0
2,2013-04-03,2018-07-01,45932,377,NaN,6552.0,8735.0,8759.0,8783.0,8759.0,4344.0
3,2013-01-23,2018-06-28,47549,474,NaN,8206.0,8759.0,8759.0,8783.0,8759.0,4283.0
4,2013-03-04,2018-07-01,46507,252,NaN,7271.0,8759.0,8759.0,8783.0,8759.0,4176.0
5,2013-01-23,2018-07-01,47230,204,NaN,8206.0,8759.0,8759.0,8783.0,8379.0,4344.0


#### test codes

In [373]:
import copy
#print the summary of datasets
d1 = copy.deepcopy(dfall_1hr_cleaned[1])
d2 = copy.deepcopy(dfall_1hr_cleaned[9])
d3 = copy.deepcopy(dfall_1hr_cleaned[11])
# nrows = len(d1)    
# new_idx = pd.date_range(d1.index[0], d1.index[nrows-1], freq='H')
# d1 = d1.reindex(new_idx)
# print len(new_idx)
# new_idx

print len(d1)
print len(d1) - pd.isnull(d1['filled']).sum()

# print pd.isnull(d1['kWh']).sum()
# d1.to_csv(os.path.join(raw_dir, 'filled.csv'))

g1 = d1.groupby(d1.index.year)
g2 = d2.groupby(d2.index.year)
g3 = d3.groupby(d3.index.year)

gg = [g1.kWh.count(), g2.kWh.count(), g3.kWh.count()]

cf = g2.kWh.count()
cf = pd.to_numeric(cf)
print type(cf)
print cf
#cf.name = '1'

df_all = pd.concat(gg, axis=1)
df_all


47610
290
<class 'pandas.core.series.Series'>
timestamp
2013    8206
2014    8759
2015    8759
2016    5040
Name: kWh, dtype: int64


,kWh,kWh,kWh
timestamp,,,
2013,8206,8206.0,8206
2014,8759,8759.0,8759
2015,8759,8759.0,8759
2016,8783,5040.0,8783
2017,8759,NaN,8759
2018,4344,NaN,4344


In [ ]:
bid = 575
d1 = dfall_1hr[bid]
d1.index = d1.index.tz_localize(None)
filename = os.path.join(processed_dir1, str(bid) + '.csv')
d1.to_csv(filename)

In [ ]:
#calculate the maximum length of consecutive values
d1 = dfall_1hr[575]
d2 = d1.kWh.isnull().astype(int).cumsum()
d3 = d1.kWh.isnull().astype(int)

#d1['int'] = d1.kWh.isnull().astype(int)
#d1['cumsum'] = d1.kWh.isnull().astype(int).cumsum()

#d1.to_csv(os.path.join(raw_dir, 'na_count.csv'))
#gb = d1['int'].groupby(d1['int'])
#gb.count()
#d3.head()
d3.head()
print len(d1)
#df = df[~d1.index.duplicated()]

In [ ]:
from itertools import groupby
for k, g in groupby(d3):
    print k, len(list(g))

d3.index[25921+2257]    
#print d3.values
#pdg = pd.groupby(d3, d3.values, group_keys =False)
#print pdg.groups[1]
#print type(pdg)
#for k,g in pdg:
#    print k, g

In [ ]:
print type(g)
for i in g:
    print i, type(i)

In [ ]:
#filling missing timeslots - 1hour interval
d1 = dfall_1hr[575]
d1.index

print len(d1)
print d1.index[len(d1)-1]
pd.date_range(d1.index[0], d1.index[len(d1)-1], )